In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers datasets sentencepiece
!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate
!pip install -q rouge-score
!pip install -q evaluate
!pip install -q torchmetrics
!pip install -q git+https://github.com/huggingface/accelerate

!git clone -q https://github.com/huggingface/transformers.git
!cd transformers
!pip install  -q -e.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 57.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 10.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR:

In [ ]:
import json
import torch
from transformers import MBart50TokenizerFast
from transformers import MBartForConditionalGeneration
# from parsivar import Normalizer
import evaluate

In [ ]:
model_name_or_path = '/content/drive/MyDrive/mBART_QA/checkpoint-1909'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = MBart50TokenizerFast.from_pretrained(model_name_or_path)
model = MBartForConditionalGeneration.from_pretrained(model_name_or_path, use_safetensors=True).to(device)

In [ ]:
metric = evaluate.load("squad")
# my_normalizer = Normalizer()

# greedy algorithm

In [ ]:
with open('/content/drive/MyDrive/dev_v9.json', 'r', encoding='utf-8-sig') as f:
  dev_data_list = json.load(f)

predicted_output = []
ground_truth = []

predicted_answers = []
theoretical_answers = []

for qa in dev_data_list:

  # ground_truth.append(my_normalizer.normalize(qa['answers'][0]['text']))
  ground_truth.append(qa['answers'][0]['text'])

  question = qa['question']
  context = qa['context']


  input_text =  f"answer: {qa['question']} context: {qa['context']} </s>"
  input_ids = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)

  attention_mask = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

  generated_answer = model.generate(input_ids, attention_mask = attention_mask, max_length=64,
                                     output_scores = True, return_dict_in_generate = True)


  generated_answers = tokenizer.batch_decode(generated_answer["sequences"], skip_special_tokens=True)
  answer_scores = generated_answer["sequences_scores"]

  answers_dict = {}
  for ans, score in zip(generated_answers, answer_scores):
    if ans and ans != '[CLS]' and ans != '[SEP]' and ans in context and len(ans) <= 30 and len(ans)>0:
        answers_dict[ans] = score
    # print(f"Answer: {ans}, Score: {score}")

  try:
    final_answer = max(answers_dict, key=answers_dict.get)
  except:
    final_answer = ''
  # predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer})
  # predicted_answers.append({"id": qa['pq_id'], "prediction_text": my_normalizer.normalize(final_answer).strip()})
  predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer.strip()})
  # predicted_output.append(my_normalizer.normalize(final_answer).strip())
  predicted_output.append(final_answer.strip())



  new_ex = {}
  new_ex['id'] = qa["pq_id"]
  new_ex['answers'] = {}
  new_ex['answers']['answer_start'] = [qa['answers'][0]['start_char']]
  new_ex['answers']['text'] = [qa['answers'][0]['text']]
  # new_ex['answers']['text'] = [my_normalizer.normalize(qa['answers'][0]['text'])]
  theoretical_answers.append(new_ex)

metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 57.13350785340314, 'f1': 60.95007571169977}

# Beam Search

In [ ]:
with open('/content/drive/MyDrive/dev_v9.json', 'r', encoding='utf-8-sig') as f:
  dev_data_list = json.load(f)

predicted_output = []
ground_truth = []

predicted_answers = []
theoretical_answers = []

for qa in dev_data_list:

  # ground_truth.append(my_normalizer.normalize(qa['answers'][0]['text']))
  ground_truth.append(qa['answers'][0]['text'])

  question = qa['question']
  context = qa['context']


  input_text =  f"answer: {qa['question']} context: {qa['context']} </s>"
  input_ids = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)

  attention_mask = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

  generated_answer = model.generate(input_ids, attention_mask = attention_mask, max_length=64, num_beams=10,
                                    output_scores = True, return_dict_in_generate = True)


  generated_answers = tokenizer.batch_decode(generated_answer["sequences"], skip_special_tokens=True)
  answer_scores = generated_answer["sequences_scores"]

  answers_dict = {}
  for ans, score in zip(generated_answers, answer_scores):
    if ans and ans != '[CLS]' and ans != '[SEP]' and ans in context and len(ans) <= 30 and len(ans)>0:
        answers_dict[ans] = score
    # print(f"Answer: {ans}, Score: {score}")

  try:
    final_answer = max(answers_dict, key=answers_dict.get)
  except:
    final_answer = ''
  # predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer})
  # predicted_answers.append({"id": qa['pq_id'], "prediction_text": my_normalizer.normalize(final_answer).strip()})
  predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer.strip()})
  # predicted_output.append(my_normalizer.normalize(final_answer).strip())
  predicted_output.append(final_answer.strip())



  new_ex = {}
  new_ex['id'] = qa["pq_id"]
  new_ex['answers'] = {}
  new_ex['answers']['answer_start'] = [qa['answers'][0]['start_char']]
  new_ex['answers']['text'] = [qa['answers'][0]['text']]
  # new_ex['answers']['text'] = [my_normalizer.normalize(qa['answers'][0]['text'])]
  theoretical_answers.append(new_ex)

metric.compute(predictions=predicted_answers, references=theoretical_answers)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'exact_match': 58.50785340314136, 'f1': 61.491213602314126}

# sampling

In [ ]:
with open('/content/drive/MyDrive/dev_v9.json', 'r', encoding='utf-8-sig') as f:
  dev_data_list = json.load(f)

predicted_output = []
ground_truth = []

predicted_answers = []
theoretical_answers = []

for qa in dev_data_list:

  # ground_truth.append(my_normalizer.normalize(qa['answers'][0]['text']))
  ground_truth.append(qa['answers'][0]['text'])

  question = qa['question']
  context = qa['context']


  input_text =  f"answer: {qa['question']} context: {qa['context']} </s>"
  input_ids = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)

  attention_mask = tokenizer(input_text,
                        return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

  generated_answer = model.generate(input_ids, attention_mask = attention_mask, max_length=64,
                                    output_scores = True, return_dict_in_generate = True,
                                    do_sample=True, top_p=1, temperature=0.95)


  generated_answers = tokenizer.batch_decode(generated_answer["sequences"], skip_special_tokens=True)
  answer_scores = generated_answer["sequences_scores"]

  answers_dict = {}
  for ans, score in zip(generated_answers, answer_scores):
    if ans and ans != '[CLS]' and ans != '[SEP]' and ans in context and len(ans) <= 30 and len(ans)>0:
        answers_dict[ans] = score
    # print(f"Answer: {ans}, Score: {score}")

  try:
    final_answer = max(answers_dict, key=answers_dict.get)
  except:
    final_answer = ''
  # predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer})
  # predicted_answers.append({"id": qa['pq_id'], "prediction_text": my_normalizer.normalize(final_answer).strip()})
  predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer.strip()})
  # predicted_output.append(my_normalizer.normalize(final_answer).strip())
  predicted_output.append(final_answer.strip())



  new_ex = {}
  new_ex['id'] = qa["pq_id"]
  new_ex['answers'] = {}
  new_ex['answers']['answer_start'] = [qa['answers'][0]['start_char']]
  new_ex['answers']['text'] = [qa['answers'][0]['text']]
  # new_ex['answers']['text'] = [my_normalizer.normalize(qa['answers'][0]['text'])]
  theoretical_answers.append(new_ex)

metric.compute(predictions=predicted_answers, references=theoretical_answers)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2674: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'exact_match': 59.94764397905759, 'f1': 62.048255455167485}